# Exemplo de uso do método RAG com LLMs, seguindo como exemplo o [video](https://www.youtube.com/watch?v=8juRvzZg_AI)

___

* Dependências

In [18]:
from pymilvus import MilvusClient, model

* Criando documentos de exemplo para o modelo utilizar como consulta

In [19]:
documents = [
    "",
    "",
    "",
    ""
]

* Fazendo o processo de embedding desses documentos
    * Esse processo transforma o texto vetores para utilizar posteriormente

In [20]:
embedding_fn = model.DefaultEmbeddingFunction()
vectors = embedding_fn.encode_documents(documents)

data = [
    {"id": i, "vector": vectors[i], "text": documents[i]}
    for i in range(len(vectors))
]

[{'id': 0, 'vector': array([ 1.59678565e-02,  4.07136710e-02, -1.51916121e-02, -2.50332078e-02,
       -2.89859072e-02,  3.65255983e-02,  4.23548855e-03, -5.35912294e-03,
        1.83380428e-02,  7.72556019e-03, -9.09491681e-02,  7.57198897e-02,
       -5.53338261e-03, -1.16566192e-02,  2.97781793e-02,  1.38699665e-03,
        5.79486090e-04, -2.44091943e-03,  9.32537318e-03,  1.25761111e-03,
       -2.37368572e-02,  1.83349437e-02,  9.24835347e-03, -3.99044261e-02,
        3.21220561e-03,  1.34891345e-02, -1.14670146e-02, -5.07589032e-03,
       -3.81472889e-02,  2.60944049e-02, -3.19338924e-02,  1.61191783e-02,
        9.64746687e-02,  1.21614825e-02, -3.22196888e-02, -2.17764625e-02,
       -4.83568957e-03,  1.24284281e-02,  6.76421645e-05, -4.24639821e-02,
        6.35287111e-02,  5.05237204e-02, -1.14835628e-02,  3.72032145e-02,
       -1.59208771e-02,  6.35767618e-03,  1.67105880e-02, -3.94518061e-02,
       -2.56469663e-02, -7.26464529e-02,  1.67844985e-02, -1.95814149e-03,
    

* Inserindo esses documentos no banco de dados vetorial Milvus

In [21]:
client = MilvusClient("basic_rag.db")

if client.has_collection(collection_name="documents"):
    client.drop_collection(collection_name="documents")

client.create_collection(collection_name="documents", dimension=768)

try:
    client.insert(collection_name="documents", data=data)
    print("insert success")
except Exception as e:
    print(e)

insert success


* Realizando uma consulta vetorial

In [22]:
query_vectors = embedding_fn.encode_queries(["Who is Alan Turing?"])

res = client.search(
    collection_name="documents",
    data=query_vectors,
    limit=2,
    output_fields=["text"],
)

print(res)

data: ["[{'id': 1, 'distance': 0.20446957647800446, 'entity': {'text': ''}}, {'id': 0, 'distance': 0.20446957647800446, 'entity': {'text': ''}}]"]
